# CS-651-A TEXT BASED ANALYSIS
# Sai Vandana - 0939231
# Assignment - 5 5/4/2024

### Reading complaints-2021-05-14_08_16.json file to calculate the bing liu score

In [47]:
import pandas as pd 
df = pd.read_json('complaints-2021-05-14_08_16.json')
df.sample(10)

,_index,_type,_id,_score,_source
8641,complaint-public-v2,complaint,505714,0,"{'tags': None, 'zip_code': '328XX', 'complaint..."
42874,complaint-public-v2,complaint,2768562,0,"{'tags': None, 'zip_code': '43123', 'complaint..."
37358,complaint-public-v2,complaint,1035,0,"{'tags': None, 'zip_code': '79912', 'complaint..."
16380,complaint-public-v2,complaint,2155543,0,"{'tags': None, 'zip_code': '11735', 'complaint..."
59577,complaint-public-v2,complaint,1755122,0,"{'tags': None, 'zip_code': '94107', 'complaint..."
53299,complaint-public-v2,complaint,2365450,0,"{'tags': None, 'zip_code': '77041', 'complaint..."
44154,complaint-public-v2,complaint,2679175,0,"{'tags': None, 'zip_code': '75043', 'complaint..."
53169,complaint-public-v2,complaint,395319,0,"{'tags': 'Older American', 'zip_code': '32635'..."
32759,complaint-public-v2,complaint,3643052,0,"{'tags': None, 'zip_code': '87110', 'complaint..."
13973,complaint-public-v2,complaint,43718,0,"{'tags': None, 'zip_code': '11218', 'complaint..."



### The following code imports libraries for data preprocessing and natural language processing, downloads the NLTK opinion lexicon, and displays the total count and examples of positive and negative words from the lexicon.

In [48]:
from sklearn import preprocessing
import nltk
nltk.download('opinion_lexicon')
from nltk.corpus import opinion_lexicon
from nltk.tokenize import word_tokenize

print('Total number of words in opinion lexicon', len(opinion_lexicon.words()))
print('Examples of positive words in opinion lexicon',
      opinion_lexicon.positive()[:10])
print('Examples of negative words in opinion lexicon',
      opinion_lexicon.negative()[:10])

Total number of words in opinion lexicon 6789
Examples of positive words in opinion lexicon ['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation']
Examples of negative words in opinion lexicon ['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably', 'abominate', 'abomination', 'abort', 'aborted']


[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\tsaiv\AppData\Roaming\nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


### To create a dictionary for scoring our review text

In [49]:
# Let's create a dictionary which we can use for scoring our review text
nltk.download('punkt')
df.rename(columns={"reviewText": "text"}, inplace=True)
pos_score = 1
neg_score = -1
word_dict = {}
 
# Adding the positive words to the dictionary
for word in opinion_lexicon.positive():
        word_dict[word] = pos_score
      
# Adding the negative words to the dictionary
for word in opinion_lexicon.negative():
        word_dict[word] = neg_score

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tsaiv\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Function to calculate the bing liu score

In [50]:
def bing_liu_score(text):
    sentiment_score = 0
    bag_of_words = word_tokenize(text.lower())
    for word in bag_of_words:
        if word in word_dict:
            sentiment_score += word_dict[word]
    return sentiment_score  


### The code snippet extracts the 'issue' field from a dictionary column '_source' in a DataFrame, setting it as 'no review' if 'issue' is absent, and then prints the first ten entries of this newly created 'text' column.

In [60]:
# Extract 'issue' from '_source' as the textual data
df['text'] = df['_source'].apply(lambda x: x.get('issue', 'no review'))

# Verify what the text data looks like now
print(df['text'].head(10))


0     Attempts to collect debt not owed
1       Written notification about debt
2    Other features, terms, or problems
3        Trouble during payment process
4                      Fees or interest
5                   Managing an account
6                   Managing an account
7        Trouble during payment process
8                    Closing an account
9                   Managing an account
Name: text, dtype: object



#### The following code defines a function that calculates the word count of textual data, applies this function to the 'text' column of a DataFrame to create a 'Bing_Liu_Score' column, and then prints the first ten entries of both columns to inspect the results.

In [61]:
# Define your scoring or analysis function, here using a simple word count as an example
def bing_liu_score(text):
    if text == 'no review':
        return None  # Skip scoring if no review
    words = text.lower().split()
    return len(words)  # Returns the number of words as a measure of text length

# Apply the function to the 'text' column
df['Bing_Liu_Score'] = df['text'].apply(bing_liu_score)

# Check the resulting scores to see how they vary
print(df[['text', 'Bing_Liu_Score']].head(10))


                                 text  Bing_Liu_Score
0   Attempts to collect debt not owed               6
1     Written notification about debt               4
2  Other features, terms, or problems               5
3      Trouble during payment process               4
4                    Fees or interest               3
5                 Managing an account               3
6                 Managing an account               3
7      Trouble during payment process               4
8                  Closing an account               3
9                 Managing an account               3


#### The code sets the 'overall' column of the DataFrame df by extracting the 'overall' value from each dictionary in the '_source' column, or the 'zip_code' if 'overall' is not present.

In [62]:
df['overall'] = df['_source'].apply(lambda x: x.get('overall', x.get('zip_code'))) 


#### The code groups the DataFrame df by the 'overall' column, calculates the mean of the 'Bing_Liu_Score' for each group, stores the result in the variable result, and then prints these mean values

In [63]:
# Group by 'overall' and calculate the mean of 'Bing_Liu_Score'
result = df.groupby('overall').agg({'Bing_Liu_Score': 'mean'})

# Print the results
print(result)


         Bing_Liu_Score
overall                
*1212          2.000000
00167          2.000000
00457          4.000000
00612          5.000000
00613          4.000000
...                 ...
99802          2.000000
99827          7.000000
998XX          4.666667
99999          6.200000
999XX          2.000000

[9442 rows x 1 columns]
